 **Media Memorability Prediction using Machine Learning**




# Introduction 

Author : **Prasad Govardhankar - 20210305**

This paper summarizes machine learning and statistical methods to predict the memorability of videos within the MediaEval Predicting Media Memorability Task. In this task, we are focused on predicting how video is memorable to viewers. Here we are predicting memorability score for videos that reflect the probability of whether a video will be remembered or not. The aim is to train computational models to get video memorability scores from visual content. Spearman’s rank correlation is used to evaluate the model through standard evaluation metrics.

In this Notebook I have used two Features on below models



1.   Extra Decision Tree
2.   Gradient Booster
3.  MLP Regressor
4. Random Forest
5. Liner Regression
6. XGB

On ***C3D, and HMP***  features seperately and with combination.


# Mounting our drive for Datasets

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Importing libraries which are require for our project



In [12]:

!pip install pyprind 
import numpy as np
import pandas as pd
#import pyprind
#from collections import Counter
#from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#from keras import preprocessing
import os
import glob

In [13]:
np.random.seed(42)

# **Load Ground Truth For Dev set**

In [14]:
# load the ground truth values
truth_path = '/content/drive/MyDrive/'
gdtruth = pd.read_csv(truth_path+'ground-truth.csv')

In [15]:
gdtruth.head(5)

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10


We don't need annotations so need to drop those columns

In [16]:
gdtruth = gdtruth.drop(columns=['nb_short-term_annotations', 'nb_long-term_annotations'])


In [17]:
gdtruth.head(5)

,video,short-term_memorability,long-term_memorability
0,video3.webm,0.924,0.846
1,video4.webm,0.923,0.667
2,video6.webm,0.863,0.700
3,video8.webm,0.922,0.818
4,video10.webm,0.950,0.900


In [18]:
gdtruth['video'] = gdtruth['video'].apply(lambda x : x.split('.')[0])

In [19]:
gdtruth.head(5)

,video,short-term_memorability,long-term_memorability
0,video3,0.924,0.846
1,video4,0.923,0.667
2,video6,0.863,0.700
3,video8,0.922,0.818
4,video10,0.950,0.900


# Load Dev C3D Features

In [20]:

C3D_feature_list= []
video_names_list = []
path = '/content/drive/MyDrive/C3D/*.txt'
for filename in glob.glob(path):
    name = ((filename.split('/')[-1]).split('.')[0])
    video_names_list.append(name) 
    with open(filename) as f:
        for line in f:
            C3D_features =[float(item) for item in line.split()]
    C3D_feature_list.append(C3D_features)



In [21]:
C3D_features_Dev = pd.DataFrame(np.array(C3D_feature_list).reshape(6000,101))
C3D_features_Dev["video"] = video_names_list

In [23]:
C3D_features_Dev.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,video
0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,7.750000e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.999985,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.120000e-06,video6633
1,0.010858,0.010386,0.000000,0.000000e+00,0.000000e+00,0.000000,2.700000e-07,0.000000,1.000000e-08,3.400000e-07,8.000000e-08,1.000000e-08,0.000004,0.000105,0.000000e+00,4.000000e-08,1.000000e-08,0.000000,0.000000,1.331800e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.600000e-07,0.000000,9.800000e-07,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,6.000000e-08,0.000021,5.000000e-08,0.000000e+00,1.000000e-08,3.000000e-08,2.000000e-08,0.000000e+00,...,0.970125,0.000016,0.001298,0.000032,0.000001,0.000000,0.000000,1.000000e-08,2.500000e-07,0.000000e+00,0.000000,5.000000e-08,0.000000e+00,1.000000e-08,1.000000e-08,4.200000e-07,0.000000e+00,3.000000e-08,2.000000e-08,0.000000,0.000000,0.000000,6.000000e-08,0.000000,0.000000e+00,9.000000e-08,0.000000e+00,0.000000e+00,1.100000e-07,1.400000e-07,0.000000,0.000000e+00,1.700000e-07,0.000000,0.000000e+00,1.000000e-08,1.300000e-06,2.600000e-06,8.000000e-08,video6632
2,0.000200,0.000065,0.993807,2.000000e-07,4.700000e-07,0.000073,3.700000e-06,0.000337,6.710000e-06,2.290000e-06,6.380000e-06,7.340000e-06,0.000019,0.000007,3.240000e-06,3.810000e-06,1.411000e-05,0.000001,0.000003,2.300000e-07,1.900000e-07,2.937620e-03,5.200000e-07,9.200000e-07,1.136000e-05,0.000011,3.310400e-04,9.200000e-07,0.000022,8.000000e-08,0.000028,0.000013,5.778000e-05,0.000037,1.683000e-05,6.860000e-06,3.990000e-06,8.030000e-06,1.450000e-06,3.800000e-07,...,0.000027,0.000013,0.000077,0.000050,0.000138,0.000012,0.000007,4.100000e-07,5.440000e-06,3.400000e-07,0.000007,8.399000e-05,4.070000e-06,3.970000e-06,5.900000e-07,1.285300e-04,8.800000e-07,2.228000e-05,1.105000e-05,0.000003,0.000015,0.000014,4.090000e-06,0.000003,4.600000e-06,4.920000e-06,5.290000e-06,6.400000e-07,2.372000e-05,6.610000e-06,0.000010,2.030000e-06,5.800000e-06,0.000001,1.490000e-06,1.170000e-05,1.500000e-07,8.300000e-07,1.060000e-04,video6634
3,0.791961,0.001496,0.006810,9.390000e-06,1.220000e-06,0.000004,7.452000e-05,0.000003,3.659000e-05,1.630000e-06,8.650000e-06,8.063000e-05,0.003490,0.000009,1.480000e-06,3.260400e-04,1.254000e-05,0.000022,0.000054,2.084000e-05,1.380000e-06,3.500000e-07,2.585000e-05,1.620000e-06,4.539700e-04,0.000008,6.030000e-06,1.475000e-05,0.000015,5.580000e-06,0.000002,0.000024,2.421700e-04,0.006136,5.600000e-06,1.200000e-07,2.860000e-06,4.289000e-05,4.110000e-06,2.330000e-06,...,0.030029,0.004600,0.000069,0.000039,0.004966,0.000006,0.000003,7.550000e-06,2.539000e-05,1.184000e-05,0.000010,6.911000e-05,6.000000e-08,2.055000e-05,1.990000e-06,8.002320e-03,7.430000e-06,1.842000e-05,3.196820e-03,0.000011,0.000256,0.000006,3.234000e-05,0.000002,6.300000e-07,3.683000e-05,1.300000e-07,7.950000e-06,4.019500e-04,2.615600e-04,0.000006,3.200000e-07,5.194400e-04,0.000002,9.700000e-07,1.735000e-05,2.574500e-04,1.305835e-01,3.993000e-05,video6639
4,0.005782,0.000306,0.004011,1.007000

In [100]:
C3D_features_Dev.to_pickle('/content/drive/MyDrive/C3D_features_Dev.pkl')

In [101]:
C3D_features_Dev = pd.read_pickle('/content/drive/MyDrive/C3D_features_Dev.pkl')

Combine Ground truth and C3D features

In [24]:
df = pd.merge(gdtruth,C3D_features_Dev)
df.shape

(6000, 104)

In [28]:
#Function to calculate Spearman coefficient scores
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s rank correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

load requried models from SKLearn

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

In [25]:
X = df.iloc[:,3:104].values
Y = df.iloc[:, 1:3].values

# Extra Decision tree on Dev C3D

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42) # random state for reproducability

extra_tree = ExtraTreeRegressor()
reg = BaggingRegressor(extra_tree).fit(X_train, Y_train)
print(reg.score(X_test,Y_test))
pred = reg.predict(X_test)
Get_score(pred, Y_test)

-0.08091399835903747
The Spearman's rank correlation coefficient is: 0.205
The Spearman's rank correlation coefficient is: 0.057


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


# MLP Regressor on Dev C3D

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20, random_state=42) # random state for reproducability
reg = MLPRegressor(random_state=1, max_iter=500)
#reg = make_pipeline(StandardScaler(),SGDRegressor(max_iter=1000, tol=1e-3))
reg.fit(X_train, Y_train)
print(reg.score(X_test,Y_test))
pred = reg.predict(X_test)
Get_score(pred, Y_test)

0.0201140114673548
The Spearman's rank correlation coefficient is: 0.287
The Spearman's rank correlation coefficient is: 0.103


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


# Gradient Booster on Dev C3D

In [31]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=10) # random state for reproducability

gbr = GradientBoostingRegressor()
model = MultiOutputRegressor(estimator=gbr)
model.fit(X_train,Y_train)
print(model.score(X_test,Y_test))
predictionsGB = model.predict(X_test)
Get_score(predictionsGB, Y_test)

0.073295338340026
The Spearman's rank correlation coefficient is: 0.319
The Spearman's rank correlation coefficient is: 0.162


# Random Forest on Dev C3D

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20, random_state=42) # random state for reproducability

reg = RandomForestRegressor(max_depth= 10,n_estimators=100)
reg.fit(X_train, Y_train)
print(reg.score(X_test,Y_test))
pred = reg.predict(X_test)
Get_score(pred, Y_test)

0.030594167920888263
The Spearman's rank correlation coefficient is: 0.291
The Spearman's rank correlation coefficient is: 0.128


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


# Linear Regression on Dev C3D

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20)

from sklearn.linear_model import LinearRegression
regressor1 = LinearRegression()
regressor1.fit(X_train, Y_train)

Y_pred1 = regressor1.predict(X_test)
Get_score(Y_pred1, Y_test)

The Spearman's rank correlation coefficient is: 0.272
The Spearman's rank correlation coefficient is: 0.112


# XGB on Dev C3D

In [34]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20, random_state=10) # random state for reproducability


xgb = xgb.XGBRegressor(learning_rate = 0.1, max_depth = 9, alpha = 10, n_estimators = 50)
model = MultiOutputRegressor(estimator=xgb)
model.fit(X_train,Y_train)
print(model.score(X_test,Y_test))
XGB = model.predict(X_test)
Get_score(XGB, Y_test)


[10:38:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[10:38:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.039817269299019464
The Spearman's rank correlation coefficient is: 0.314
The Spearman's rank correlation coefficient is: 0.092


I've tried multiple models and as we can see **Gradient Boost Regressor** out performed other models.

# Now we'll try same Models with HMP feature

In [35]:

def load_hmp(hmp_path):
    files = list(gdtruth["video"].values)
    hmp_features = []
    video_names_list = []
    for file in files:
        file = hmp_path+file+'.txt'
        name = ((file.split('/')[-1]).split('.')[0])
        with open(file) as f:
            for line in f:
                pairs=line.split()
                HMP_temp = { int(p.split(':')[0]) : float(p.split(':')[1]) for p in pairs}
                HMP = np.zeros(6075)
            for idx in HMP_temp.keys():
                HMP[idx-1] = HMP_temp[idx]
            hmp_features.append(HMP)
            video_names_list.append(name)
    return hmp_features

# Load HMP Dev Set

In [36]:
hmp_path = '/content/drive/MyDrive/HMP/'

hmp_features = load_hmp(hmp_path)

In [37]:
len(hmp_features)

6000

In [ ]:
print(video_names_list)

In [38]:
HMP_features = pd.DataFrame(np.array(hmp_features).reshape(6000,6075))
HMP_features["video"] = video_names_list
HMP_features.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,6036,6037,6038,6039,6040,6041,6042,6043,6044,6045,6046,6047,6048,6049,6050,6051,6052,6053,6054,6055,6056,6057,6058,6059,6060,6061,6062,6063,6064,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074,video
0,0.125563,0.024036,0.000314,0.0,0.015864,0.000358,0.0,0.000000,0.000086,0.0,0.0,0.0,0.0,0.002795,0.000054,0.0,0.000000,0.000037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000173,0.000459,0.000000,0.000148,0.000104,0.000000,0.000121,0.000551,0.000000,0.000114,0.000884,0.000002,0.000116,0.000077,0.000002,0.000027,0.000136,0.000000,0.000000,0.000002,0.000000,0.000091,0.000035,0.000000,0.000163,0.000467,0.000002,0.000010,0.000017,0.000000,0.000393,0.000279,0.000000,0.000289,0.001926,0.000000,0.000086,0.000580,0.000000,video6633
1,0.007526,0.001421,0.000068,0.0,0.001184,0.000143,0.0,0.000000,0.000079,0.0,0.0,0.0,0.0,0.000246,0.000024,0.0,0.000000,0.000042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000308,0.001054,0.000751,0.000176,0.000062,0.000000,0.000123,0.000398,0.000086,0.000246,0.000433,0.000446,0.000143,0.000053,0.000000,0.000053,0.000099,0.000009,0.000004,0.000033,0.000004,0.000051,0.000035,0.000000,0.000062,0.000358,0.000035,0.000024,0.000083,0.000053,0.000244,0.000066,0.000000,0.000081,0.000617,0.000094,0.000220,0.000762,0.001224,video6632
2,0.109584,0.018978,0.000289,0.0,0.008774,0.000208,0.0,0.000002,0.000088,0.0,0.0,0.0,0.0,0.002046,0.000061,0.0,0.000000,0.000038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000057,0.000158,0.000073,0.000021,0.000009,0.000002,0.000019,0.000095,0.000021,0.000019,0.000090,0.000073,0.000050,0.000024,0.000000,0.000012,0.000021,0.000000,0.000000,0.000002,0.000000,0.000017,0.000007,0.000002,0.000066,0.000203,0.000026,0.000002,0.000040,0.000007,0.000054,0.000045,0.000000,0.000028,0.000291,0.000033,0.000052,0.000258,0.000215,video6634
3,0.120431,0.013561,0.000277,0.0,0.018974,0.000913,0.0,0.000024,0.000713,0.0,0.0,0.0,0.0,0.002496,0.000149,0.0,0.000011,0.000157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000434,0.000543,0.000412,0.000412,0.000045,0.000003,0.000144,0.000282,0.000037,0.000197,0.000218,0.000157,0.000237,0.000021,0.000000,0.000040,0.000056,0.000008,0.000005,0.000013,0.000019,0.000168,0.000013,0.000000,0.000133,0.000202,0.000029,0.000029,0.000035,0.000059,0.001110,0.000075,0.000008,0.000333,0.000793,0.000101,0.000588,0.000503,0.000452,video6639
4,0.005026,0.001356,0.000055,0.0,0.000665,0.000029,0.0,0.000000,0.000024,0.0,0.0,0.0,0.0,0.000147,0.000020,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000996,0.001604,0.000103,0.000768,0.000215,0.000009,0.000415,0.000926,0.000020,0.000538,0.001178,0.000050,0.000518,0.000169,0.000007,0.000134,0.000169,0.000007,0.000026,0.000046,0.000007,0.000373,0.000088,0.000000,0.000338,0.000441,0.000029,0.000070,0.000149,0.000009,0.000882,0.000200,0.000009,0.000559,0.001097,0.000018,0.000632,0.001128,0.000064,video6643


In [ ]:
#HMP_features = pd.merge(gdtruth,HMP_features)

In [39]:
HMP_features.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,6036,6037,6038,6039,6040,6041,6042,6043,6044,6045,6046,6047,6048,6049,6050,6051,6052,6053,6054,6055,6056,6057,6058,6059,6060,6061,6062,6063,6064,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074,video
0,0.125563,0.024036,0.000314,0.0,0.015864,0.000358,0.0,0.000000,0.000086,0.0,0.0,0.0,0.0,0.002795,0.000054,0.0,0.000000,0.000037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000173,0.000459,0.000000,0.000148,0.000104,0.000000,0.000121,0.000551,0.000000,0.000114,0.000884,0.000002,0.000116,0.000077,0.000002,0.000027,0.000136,0.000000,0.000000,0.000002,0.000000,0.000091,0.000035,0.000000,0.000163,0.000467,0.000002,0.000010,0.000017,0.000000,0.000393,0.000279,0.000000,0.000289,0.001926,0.000000,0.000086,0.000580,0.000000,video6633
1,0.007526,0.001421,0.000068,0.0,0.001184,0.000143,0.0,0.000000,0.000079,0.0,0.0,0.0,0.0,0.000246,0.000024,0.0,0.000000,0.000042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000308,0.001054,0.000751,0.000176,0.000062,0.000000,0.000123,0.000398,0.000086,0.000246,0.000433,0.000446,0.000143,0.000053,0.000000,0.000053,0.000099,0.000009,0.000004,0.000033,0.000004,0.000051,0.000035,0.000000,0.000062,0.000358,0.000035,0.000024,0.000083,0.000053,0.000244,0.000066,0.000000,0.000081,0.000617,0.000094,0.000220,0.000762,0.001224,video6632
2,0.109584,0.018978,0.000289,0.0,0.008774,0.000208,0.0,0.000002,0.000088,0.0,0.0,0.0,0.0,0.002046,0.000061,0.0,0.000000,0.000038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000057,0.000158,0.000073,0.000021,0.000009,0.000002,0.000019,0.000095,0.000021,0.000019,0.000090,0.000073,0.000050,0.000024,0.000000,0.000012,0.000021,0.000000,0.000000,0.000002,0.000000,0.000017,0.000007,0.000002,0.000066,0.000203,0.000026,0.000002,0.000040,0.000007,0.000054,0.000045,0.000000,0.000028,0.000291,0.000033,0.000052,0.000258,0.000215,video6634
3,0.120431,0.013561,0.000277,0.0,0.018974,0.000913,0.0,0.000024,0.000713,0.0,0.0,0.0,0.0,0.002496,0.000149,0.0,0.000011,0.000157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000434,0.000543,0.000412,0.000412,0.000045,0.000003,0.000144,0.000282,0.000037,0.000197,0.000218,0.000157,0.000237,0.000021,0.000000,0.000040,0.000056,0.000008,0.000005,0.000013,0.000019,0.000168,0.000013,0.000000,0.000133,0.000202,0.000029,0.000029,0.000035,0.000059,0.001110,0.000075,0.000008,0.000333,0.000793,0.000101,0.000588,0.000503,0.000452,video6639
4,0.005026,0.001356,0.000055,0.0,0.000665,0.000029,0.0,0.000000,0.000024,0.0,0.0,0.0,0.0,0.000147,0.000020,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000996,0.001604,0.000103,0.000768,0.000215,0.000009,0.000415,0.000926,0.000020,0.000538,0.001178,0.000050,0.000518,0.000169,0.000007,0.000134,0.000169,0.000007,0.000026,0.000046,0.000007,0.000373,0.000088,0.000000,0.000338,0.000441,0.000029,0.000070,0.000149,0.000009,0.000882,0.000200,0.000009,0.000559,0.001097,0.000018,0.000632,0.001128,0.000064,video6643


In [102]:
HMP_features.to_pickle('/content/drive/MyDrive/HMP_features.pkl')

In [103]:
HMP_features = pd.read_pickle('/content/drive/MyDrive/HMP_features.pkl')

In [41]:
X = HMP_features.iloc[:,0:6075].values
Y = gdtruth[['short-term_memorability','long-term_memorability']].values

# EXTRA Decision Tree on Dev HMP

In [42]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42) # random state for reproducability

extra_tree = ExtraTreeRegressor()
reg = BaggingRegressor(extra_tree).fit(X_train, Y_train)
print(reg.score(X_test,Y_test))
pred = reg.predict(X_test)
Get_score(pred, Y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


-0.06009090924388922
The Spearman's rank correlation coefficient is: 0.206
The Spearman's rank correlation coefficient is: 0.046


# Gradient booster on Dev HMP

In [43]:


from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42) # random state for reproducability

gbr = GradientBoostingRegressor()
model = MultiOutputRegressor(estimator=gbr)
model.fit(X_train,Y_train)
print(model.score(X_test,Y_test))
predictionsGB = model.predict(X_test)
Get_score(predictionsGB, Y_test)

0.04894695876668437
The Spearman's rank correlation coefficient is: 0.310
The Spearman's rank correlation coefficient is: 0.113


# MLP REGRESSOR on Dev HMP

In [44]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20, random_state=42) # random state for reproducability
reg = MLPRegressor(random_state=1, max_iter=500)
#reg = make_pipeline(StandardScaler(),SGDRegressor(max_iter=1000, tol=1e-3))
reg.fit(X_train, Y_train)
print(reg.score(X_test,Y_test))
pred = reg.predict(X_test)
Get_score(pred, Y_test)

0.017501924837915333
The Spearman's rank correlation coefficient is: 0.257
The Spearman's rank correlation coefficient is: 0.111


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


# Random Forest on Dev HMP

In [45]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20, random_state=42) # random state for reproducability

reg = RandomForestRegressor(max_depth= 20,n_estimators=100)
reg.fit(X_train, Y_train)
print(reg.score(X_test,Y_test))
pred = reg.predict(X_test)
Get_score(pred, Y_test)

0.03279023738417329
The Spearman's rank correlation coefficient is: 0.315
The Spearman's rank correlation coefficient is: 0.128


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


# Linear Regression on Dev HMP

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20)

from sklearn.linear_model import LinearRegression
regressor1 = LinearRegression()
regressor1.fit(X_train, Y_train)

Y_pred1 = regressor1.predict(X_test)
Get_score(Y_pred1, Y_test)

The Spearman's rank correlation coefficient is: 0.009
The Spearman's rank correlation coefficient is: 0.073


# XGB on Dev HMP

In [47]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20, random_state=42) # random state for reproducability


xgb = xgb.XGBRegressor(learning_rate = 0.2, max_depth = 5, alpha = 10, n_estimators = 20)
model = MultiOutputRegressor(estimator=xgb)
model.fit(X_train,Y_train)
print(model.score(X_test,Y_test))
XGB = model.predict(X_test)
Get_score(XGB, Y_test)

/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[11:50:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:51:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.024981902128770994
The Spearman's rank correlation coefficient is: 0.292
The Spearman's rank correlation coefficient is: 0.092


# **Executing same models for combination of C3D and hmp combined Feature!**


merge C3D + HMP + Ground truth

In [50]:
len(C3D_features_Dev)

6000

In [49]:
len(HMP_features)

6000

In [51]:
#Combining captions and c3d with HMP
c3d_hmp  = pd.concat([C3D_features_Dev,HMP_features],axis=1)


In [52]:
len(c3d_hmp)

6000

In [53]:
c3d_hmp.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,6036,6037,6038,6039,6040,6041,6042,6043,6044,6045,6046,6047,6048,6049,6050,6051,6052,6053,6054,6055,6056,6057,6058,6059,6060,6061,6062,6063,6064,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074,video
0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,7.750000e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000173,0.000459,0.000000,0.000148,0.000104,0.000000,0.000121,0.000551,0.000000,0.000114,0.000884,0.000002,0.000116,0.000077,0.000002,0.000027,0.000136,0.000000,0.000000,0.000002,0.000000,0.000091,0.000035,0.000000,0.000163,0.000467,0.000002,0.000010,0.000017,0.000000,0.000393,0.000279,0.000000,0.000289,0.001926,0.000000,0.000086,0.000580,0.000000,video6633
1,0.010858,0.010386,0.000000,0.000000e+00,0.000000e+00,0.000000,2.700000e-07,0.000000,1.000000e-08,3.400000e-07,8.000000e-08,1.000000e-08,0.000004,0.000105,0.000000e+00,4.000000e-08,1.000000e-08,0.000000e+00,0.000000,1.331800e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.600000e-07,0.000000,9.800000e-07,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,6.000000e-08,0.000021,5.000000e-08,0.000000e+00,1.000000e-08,3.000000e-08,2.000000e-08,0.000000e+00,...,0.000308,0.001054,0.000751,0.000176,0.000062,0.000000,0.000123,0.000398,0.000086,0.000246,0.000433,0.000446,0.000143,0.000053,0.000000,0.000053,0.000099,0.000009,0.000004,0.000033,0.000004,0.000051,0.000035,0.000000,0.000062,0.000358,0.000035,0.000024,0.000083,0.000053,0.000244,0.000066,0.000000,0.000081,0.000617,0.000094,0.000220,0.000762,0.001224,video6632
2,0.000200,0.000065,0.993807,2.000000e-07,4.700000e-07,0.000073,3.700000e-06,0.000337,6.710000e-06,2.290000e-06,6.380000e-06,7.340000e-06,0.000019,0.000007,3.240000e-06,3.810000e-06,1.411000e-05,1.040000e-06,0.000003,2.300000e-07,1.900000e-07,2.937620e-03,5.200000e-07,9.200000e-07,1.136000e-05,0.000011,3.310400e-04,9.200000e-07,0.000022,8.000000e-08,0.000028,0.000013,5.778000e-05,0.000037,1.683000e-05,6.860000e-06,3.990000e-06,8.030000e-06,1.450000e-06,3.800000e-07,...,0.000057,0.000158,0.000073,0.000021,0.000009,0.000002,0.000019,0.000095,0.000021,0.000019,0.000090,0.000073,0.000050,0.000024,0.000000,0.000012,0.000021,0.000000,0.000000,0.000002,0.000000,0.000017,0.000007,0.000002,0.000066,0.000203,0.000026,0.000002,0.000040,0.000007,0.000054,0.000045,0.000000,0.000028,0.000291,0.000033,0.000052,0.000258,0.000215,video6634
3,0.791961,0.001496,0.006810,9.390000e-06,1.220000e-06,0.000004,7.452000e-05,0.000003,3.659000e-05,1.630000e-06,8.650000e-06,8.063000e-05,0.003490,0.000009,1.480000e-06,3.260400e-04,1.254000e-05,2.197000e-05,0.000054,2.084000e-05,1.380000e-06,3.500000e-07,2.585000e-05,1.620000e-06,4.539700e-04,0.000008,6.030000e-06,1.475000e-05,0.000015,5.580000e-06,0.000002,0.000024,2.421700e-04,0.006136,5.600000e-06,1.200000e-07,2.860000e-06,4.289000e-05,4.110000e-06,2.330000e-06,...,0.000434,0.000543,0.000412,0.000412,0.000045,0.000003,0.000144,0.000282,0.000037,0.000197,0.000218,0.000157,0.000237,0.000021,0.000000,0.000040,0.000056,0.000008,0.000005,0.000013,0.000019,0.000168,0.000013,0.000000,0.000133,0.000202,0.000029,0.000029,0.000035,0.000059,0.001110,0.000075,0.000008,0.000333,0.000793,0.000101,0.000588,0.000503,0.000452,video6639
4,0.005782,0.000306,0.004011,1.007000e-05,1.034000e-05,0.000002,3.160000e-06,0.000003,1.984000e-05,5.750000e-06,6.642000e-05,6.690000e-06,0.000301,0.004799,2.800000e-07,1.669000e-05,2.670000e-06,1.060000e-06,0.000011,6.526000e-05,3.100000e-07,3.590000e-05,1.400000e-06,6.260000e-06,1.750103e-02,0.000019,1.190515e-02,4.450000e-06,0.000005,2.387

In [107]:
c3d_hmp.shape

(6000, 6178)

In [108]:
X = c3d_hmp.drop(columns=['video'])
Y = gdtruth[['short-term_memorability','long-term_memorability']].values

# EXTRA Decision tree ON C3D and HMP

In [56]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42) # random state for reproducability

extra_tree = ExtraTreeRegressor()
reg = BaggingRegressor(extra_tree).fit(X_train, Y_train)
print(reg.score(X_test,Y_test))
pred = reg.predict(X_test)
Get_score(pred, Y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


-0.07250704061438354
The Spearman's rank correlation coefficient is: 0.160
The Spearman's rank correlation coefficient is: 0.066


# Gradient Booster ON C3D and HMP

In [110]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42) # random state for reproducability

gbr = GradientBoostingRegressor()
model = MultiOutputRegressor(estimator=gbr)
model.fit(X_train,Y_train)
print(model.score(X_test,Y_test))
predictionsGB = model.predict(X_test)
Get_score(predictionsGB, Y_test)

0.04428595301461358
The Spearman's rank correlation coefficient is: 0.299
The Spearman's rank correlation coefficient is: 0.117


# MLREGRESSOR ON C3D and HMP

In [58]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20, random_state=42) # random state for reproducability
reg = MLPRegressor(random_state=1, max_iter=500)
#reg = make_pipeline(StandardScaler(),SGDRegressor(max_iter=1000, tol=1e-3))
reg.fit(X_train, Y_train)
print(reg.score(X_test,Y_test))
pred = reg.predict(X_test)
Get_score(pred, Y_test)

0.006162197509033656
The Spearman's rank correlation coefficient is: 0.199
The Spearman's rank correlation coefficient is: 0.082


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


# Random Forest ON C3D and HMP

In [59]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20, random_state=42) # random state for reproducability

reg = RandomForestRegressor(max_depth= 20,n_estimators=100)
reg.fit(X_train, Y_train)
print(reg.score(X_test,Y_test))
pred = reg.predict(X_test)
Get_score(pred, Y_test)

0.029278291877852566
The Spearman's rank correlation coefficient is: 0.275
The Spearman's rank correlation coefficient is: 0.126


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


# Liner Regression ON C3D and HMP

In [60]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20)

from sklearn.linear_model import LinearRegression
regressor1 = LinearRegression()
regressor1.fit(X_train, Y_train)

Y_pred1 = regressor1.predict(X_test)
Get_score(Y_pred1, Y_test)

The Spearman's rank correlation coefficient is: 0.010
The Spearman's rank correlation coefficient is: 0.002


# XGB ON C3D and HMP

In [61]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20, random_state=42) # random state for reproducability


xgb = xgb.XGBRegressor(learning_rate = 0.2, max_depth = 5, alpha = 10, n_estimators = 20)
model = MultiOutputRegressor(estimator=xgb)
model.fit(X_train,Y_train)
print(model.score(X_test,Y_test))
XGB = model.predict(X_test)
Get_score(XGB, Y_test)

/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[12:47:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:48:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.02983055559527381
The Spearman's rank correlation coefficient is: 0.273
The Spearman's rank correlation coefficient is: 0.101


# Executing Best performing model on the Test Dataset

#Load C3D test set

In [63]:
os.chdir('/content/drive/MyDrive/C3D_test')

In [64]:
C3D_feature_Test_list= []
video_names_Test_list = []
path = '/content/drive/MyDrive/C3D_test/*.txt'
for filename in glob.glob(path):
    name = ((filename.split('/')[-1]).split('.')[0])
    video_names_Test_list.append(name) 
    with open(filename) as f:
        for line in f:
            C3D_features_Test =[float(item) for item in line.split()]
    C3D_feature_Test_list.append(C3D_features_Test)

In [65]:
C3D_features_Test = pd.DataFrame(np.array(C3D_feature_Test_list).reshape(2000,101))
C3D_features_Test["video"] = video_names_Test_list
C3D_features_Test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,video
0,0.968162,0.001538,1.700000e-07,3.000000e-08,0.000000,0.000000,3.000000e-08,0.000000,0.000000,1.700000e-07,1.000000e-08,0.000000,0.018083,0.000007,0.000000,4.000000e-08,3.000000e-08,0.000002,1.000000e-08,0.000096,0.000000,0.000000,0.000000e+00,0.000000,5.400000e-07,0.000000,2.900000e-07,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.000000e-08,0.007307,1.300000e-07,0.000000,1.700000e-07,0.000000,0.000201,0.000000,...,0.000991,0.000007,0.000062,0.000002,0.000006,0.000000e+00,2.000000e-08,6.000000e-08,2.000000e-08,1.000000e-08,1.000000e-08,1.200000e-07,0.000000,0.000000,0.000000,0.002485,0.000000,0.000000,0.000000,0.000000,2.000000e-08,0.000000,1.000000e-08,0.000000e+00,0.00000,0.000000,0.000000,1.000000e-08,1.000000e-08,1.000000e-07,0.000000,0.000000,0.000005,0.000000,0.000000e+00,0.000000,0.000007,0.000002,0.000001,video8770
1,0.000000,0.000000,0.000000e+00,0.000000e+00,1.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,1.000000e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000e-08,1.000000e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,2.800000e-07,0.00000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,video8760
2,0.000358,0.003973,8.803300e-03,7.740530e-03,0.000403,0.000194,9.094740e-03,0.017529,0.000285,2.582000e-05,3.279300e-04,0.013337,0.000007,0.000169,0.000122,6.851600e-04,8.164600e-04,0.000004,1.249030e-03,0.000297,0.001731,0.017458,4.575800e-04,0.011841,6.208798e-02,0.000916,3.288000e-05,0.001933,0.001350,3.050290e-03,0.000202,0.138158,2.006050e-03,0.000027,1.112671e-02,0.028939,7.646010e-03,0.015354,0.006985,0.000424,...,0.000889,0.001144,0.003669,0.006591,0.000239,4.588470e-03,3.227340e-03,4.836100e-04,9.350500e-04,6.927425e-02,4.976600e-04,1.659992e-02,0.000226,0.086639,0.000103,0.001129,0.000026,0.004527,0.003472,0.045509,6.679720e-03,0.001158,1.091790e-03,1.939896e-02,0.00162,0.004252,0.003154,5.585500e-04,1.479572e-02,1.822863e-02,0.058241,0.002662,0.028245,0.000521,5.476000e-05,0.002431,0.000706,0.045552,0.001591,video8765
3,0.001573,0.001398,7.172299e-02,2.488000e-05,0.000019,0.000827,6.838070e-03,0.000136,0.000029,2.393100e-04,5.657900e-04,0.000672,0.000265,0.000454,0.000013,3.156000e-04,5.110800e-04,0.000008,9.750000e-06,0.000004,0.000010,0.001801,1.660000e-06,0.000047,4.602800e-04,0.000031,4.249600e-04,0.000133,0.001446,2.483600e-04,0.000171,0.000055,6.298570e-03,0.000264,5.318000e-05,0.000003,8.437000e-05,0.000372,0.000109,0.000013,...,0.003979,0.003753,0.002594,0.003642,0.030717,3.481370e-03,5.950400e-04,1.133000e-05,5.056500e-04,1.954000e-04,5.344000e-05,3.529600e-04,0.000009,0.000182,0.000090,0.000233,0.001900,0.000345,0.000182,0.000258,1.013890e-03,0.000054,6.851000e-05,1.925300e-04,0.00141,0.000869,0.000014,8.568000e-05,1.698460e-03,4.608000e-05,0.001467,0.000128,0.000102,0.000007,3.577000e-05,0.001544,0.000034,0.000837,0.804485,video8762
4,0.003526,0.001376,4.618705e-02,7.760000e-06,0.000041,0.000295,3.431900e-04,0.000406,0.000070,1.534000e-05,3.539800e-04,0.000599,0.000349,0.004517,0.000016,5.500000e-05,5.680000e-05,0.000015,1.409000e-05,0.000145,0.000004,0.153209,6.900000e-07,0.000029,2.325810e-03,0.000133,2.068100e-03,0.000012,0.000026,1.545800e-04,0.000009,0.000481,1.428600e-04,0.000359,1.008580e-03,0.000114,4.409000e-0

In [66]:
C3D_features_Test = C3D_features_Test.drop(['video'], axis=1)

# Combine C3D Train and Test Features

In [67]:
C3D_final_dataset = pd.concat([C3D_features_Dev, C3D_features_Test],ignore_index=True)
C3D_final_dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,video
0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,7.750000e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.999985,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.120000e-06,video6633
1,0.010858,0.010386,0.000000,0.000000e+00,0.000000e+00,0.000000,2.700000e-07,0.000000,1.000000e-08,3.400000e-07,8.000000e-08,1.000000e-08,0.000004,0.000105,0.000000e+00,4.000000e-08,1.000000e-08,0.000000,0.000000,1.331800e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.600000e-07,0.000000,9.800000e-07,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,6.000000e-08,0.000021,5.000000e-08,0.000000e+00,1.000000e-08,3.000000e-08,2.000000e-08,0.000000e+00,...,0.970125,0.000016,0.001298,0.000032,0.000001,0.000000,0.000000,1.000000e-08,2.500000e-07,0.000000e+00,0.000000,5.000000e-08,0.000000e+00,1.000000e-08,1.000000e-08,4.200000e-07,0.000000e+00,3.000000e-08,2.000000e-08,0.000000,0.000000,0.000000,6.000000e-08,0.000000,0.000000e+00,9.000000e-08,0.000000e+00,0.000000e+00,1.100000e-07,1.400000e-07,0.000000,0.000000e+00,1.700000e-07,0.000000,0.000000e+00,1.000000e-08,1.300000e-06,2.600000e-06,8.000000e-08,video6632
2,0.000200,0.000065,0.993807,2.000000e-07,4.700000e-07,0.000073,3.700000e-06,0.000337,6.710000e-06,2.290000e-06,6.380000e-06,7.340000e-06,0.000019,0.000007,3.240000e-06,3.810000e-06,1.411000e-05,0.000001,0.000003,2.300000e-07,1.900000e-07,2.937620e-03,5.200000e-07,9.200000e-07,1.136000e-05,0.000011,3.310400e-04,9.200000e-07,0.000022,8.000000e-08,0.000028,0.000013,5.778000e-05,0.000037,1.683000e-05,6.860000e-06,3.990000e-06,8.030000e-06,1.450000e-06,3.800000e-07,...,0.000027,0.000013,0.000077,0.000050,0.000138,0.000012,0.000007,4.100000e-07,5.440000e-06,3.400000e-07,0.000007,8.399000e-05,4.070000e-06,3.970000e-06,5.900000e-07,1.285300e-04,8.800000e-07,2.228000e-05,1.105000e-05,0.000003,0.000015,0.000014,4.090000e-06,0.000003,4.600000e-06,4.920000e-06,5.290000e-06,6.400000e-07,2.372000e-05,6.610000e-06,0.000010,2.030000e-06,5.800000e-06,0.000001,1.490000e-06,1.170000e-05,1.500000e-07,8.300000e-07,1.060000e-04,video6634
3,0.791961,0.001496,0.006810,9.390000e-06,1.220000e-06,0.000004,7.452000e-05,0.000003,3.659000e-05,1.630000e-06,8.650000e-06,8.063000e-05,0.003490,0.000009,1.480000e-06,3.260400e-04,1.254000e-05,0.000022,0.000054,2.084000e-05,1.380000e-06,3.500000e-07,2.585000e-05,1.620000e-06,4.539700e-04,0.000008,6.030000e-06,1.475000e-05,0.000015,5.580000e-06,0.000002,0.000024,2.421700e-04,0.006136,5.600000e-06,1.200000e-07,2.860000e-06,4.289000e-05,4.110000e-06,2.330000e-06,...,0.030029,0.004600,0.000069,0.000039,0.004966,0.000006,0.000003,7.550000e-06,2.539000e-05,1.184000e-05,0.000010,6.911000e-05,6.000000e-08,2.055000e-05,1.990000e-06,8.002320e-03,7.430000e-06,1.842000e-05,3.196820e-03,0.000011,0.000256,0.000006,3.234000e-05,0.000002,6.300000e-07,3.683000e-05,1.300000e-07,7.950000e-06,4.019500e-04,2.615600e-04,0.000006,3.200000e-07,5.194400e-04,0.000002,9.700000e-07,1.735000e-05,2.574500e-04,1.305835e-01,3.993000e-05,video6639
4,0.005782,0.000306,0.004011,1.007000

In [68]:
C3D_final_dataset = C3D_final_dataset.drop(['video'], axis=1)

In [69]:
C3D_final_dataset.shape

(8000, 101)

# Load HMP TEST DATA

In [70]:
def read_HMP(fname):
    """Scan HMP(Histogram of Motion Patterns) features from file"""
    with open(fname) as f:
        for line in f:
            pairs=line.split()
            HMP_temp = { int(p.split(':')[0]) : float(p.split(':')[1]) for p in pairs}
    # there are 6075 bins, fill zeros
    HMP = np.zeros(6075)
    for idx in HMP_temp.keys():
        HMP[idx-1] = HMP_temp[idx]            
    return HMP

In [71]:
HMP_feature_list_test= []
video_names_list_test = []
for filename in glob.glob('/content/drive/MyDrive/HMP_test/*.txt'):
    name = ((filename.split('/')[-1]).split('.')[0]+'.webm')
    video_names_list.append(name)
    HMP_features_test = read_HMP(filename)
    HMP_feature_list_test.append(HMP_features_test)

HMP_features_Test = pd.DataFrame(np.array(HMP_feature_list_test).reshape(2000,6075))


In [72]:
HMP_features_Test.shape

(2000, 6075)

Combine HMP train and test set

In [73]:
HMP_final_dataset = pd.concat([HMP_features, HMP_features_Test],ignore_index=True)
HMP_final_dataset.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,6036,6037,6038,6039,6040,6041,6042,6043,6044,6045,6046,6047,6048,6049,6050,6051,6052,6053,6054,6055,6056,6057,6058,6059,6060,6061,6062,6063,6064,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074,video
0,0.125563,0.024036,0.000314,0.0,0.015864,0.000358,0.0,0.000000,0.000086,0.0,0.0,0.0,0.0,0.002795,0.000054,0.0,0.000000,0.000037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000173,0.000459,0.000000,0.000148,0.000104,0.000000,0.000121,0.000551,0.000000,0.000114,0.000884,0.000002,0.000116,0.000077,0.000002,0.000027,0.000136,0.000000,0.000000,0.000002,0.000000,0.000091,0.000035,0.000000,0.000163,0.000467,0.000002,0.000010,0.000017,0.000000,0.000393,0.000279,0.000000,0.000289,0.001926,0.000000,0.000086,0.000580,0.000000,video6633
1,0.007526,0.001421,0.000068,0.0,0.001184,0.000143,0.0,0.000000,0.000079,0.0,0.0,0.0,0.0,0.000246,0.000024,0.0,0.000000,0.000042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000308,0.001054,0.000751,0.000176,0.000062,0.000000,0.000123,0.000398,0.000086,0.000246,0.000433,0.000446,0.000143,0.000053,0.000000,0.000053,0.000099,0.000009,0.000004,0.000033,0.000004,0.000051,0.000035,0.000000,0.000062,0.000358,0.000035,0.000024,0.000083,0.000053,0.000244,0.000066,0.000000,0.000081,0.000617,0.000094,0.000220,0.000762,0.001224,video6632
2,0.109584,0.018978,0.000289,0.0,0.008774,0.000208,0.0,0.000002,0.000088,0.0,0.0,0.0,0.0,0.002046,0.000061,0.0,0.000000,0.000038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000057,0.000158,0.000073,0.000021,0.000009,0.000002,0.000019,0.000095,0.000021,0.000019,0.000090,0.000073,0.000050,0.000024,0.000000,0.000012,0.000021,0.000000,0.000000,0.000002,0.000000,0.000017,0.000007,0.000002,0.000066,0.000203,0.000026,0.000002,0.000040,0.000007,0.000054,0.000045,0.000000,0.000028,0.000291,0.000033,0.000052,0.000258,0.000215,video6634
3,0.120431,0.013561,0.000277,0.0,0.018974,0.000913,0.0,0.000024,0.000713,0.0,0.0,0.0,0.0,0.002496,0.000149,0.0,0.000011,0.000157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000434,0.000543,0.000412,0.000412,0.000045,0.000003,0.000144,0.000282,0.000037,0.000197,0.000218,0.000157,0.000237,0.000021,0.000000,0.000040,0.000056,0.000008,0.000005,0.000013,0.000019,0.000168,0.000013,0.000000,0.000133,0.000202,0.000029,0.000029,0.000035,0.000059,0.001110,0.000075,0.000008,0.000333,0.000793,0.000101,0.000588,0.000503,0.000452,video6639
4,0.005026,0.001356,0.000055,0.0,0.000665,0.000029,0.0,0.000000,0.000024,0.0,0.0,0.0,0.0,0.000147,0.000020,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000996,0.001604,0.000103,0.000768,0.000215,0.000009,0.000415,0.000926,0.000020,0.000538,0.001178,0.000050,0.000518,0.000169,0.000007,0.000134,0.000169,0.000007,0.000026,0.000046,0.000007,0.000373,0.000088,0.000000,0.000338,0.000441,0.000029,0.000070,0.000149,0.000009,0.000882,0.000200,0.000009,0.000559,0.001097,0.000018,0.000632,0.001128,0.000064,video6643


In [74]:
HMP_final_dataset.shape

(8000, 6076)

Combine Train and Test

In [75]:
c3d_hmp_Final  = pd.concat([C3D_final_dataset,HMP_final_dataset],axis=1)

In [76]:
c3d_hmp_Final.shape

(8000, 6177)

In [77]:
c3d_hmp_Final = c3d_hmp_Final.drop(['video'], axis=1)

In [78]:
c3d_hmp_Final.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,6035,6036,6037,6038,6039,6040,6041,6042,6043,6044,6045,6046,6047,6048,6049,6050,6051,6052,6053,6054,6055,6056,6057,6058,6059,6060,6061,6062,6063,6064,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074
0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,7.750000e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000002,0.000173,0.000459,0.000000,0.000148,0.000104,0.000000,0.000121,0.000551,0.000000,0.000114,0.000884,0.000002,0.000116,0.000077,0.000002,0.000027,0.000136,0.000000,0.000000,0.000002,0.000000,0.000091,0.000035,0.000000,0.000163,0.000467,0.000002,0.000010,0.000017,0.000000,0.000393,0.000279,0.000000,0.000289,0.001926,0.000000,0.000086,0.000580,0.000000
1,0.010858,0.010386,0.000000,0.000000e+00,0.000000e+00,0.000000,2.700000e-07,0.000000,1.000000e-08,3.400000e-07,8.000000e-08,1.000000e-08,0.000004,0.000105,0.000000e+00,4.000000e-08,1.000000e-08,0.000000e+00,0.000000,1.331800e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.600000e-07,0.000000,9.800000e-07,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,6.000000e-08,0.000021,5.000000e-08,0.000000e+00,1.000000e-08,3.000000e-08,2.000000e-08,0.000000e+00,...,0.000685,0.000308,0.001054,0.000751,0.000176,0.000062,0.000000,0.000123,0.000398,0.000086,0.000246,0.000433,0.000446,0.000143,0.000053,0.000000,0.000053,0.000099,0.000009,0.000004,0.000033,0.000004,0.000051,0.000035,0.000000,0.000062,0.000358,0.000035,0.000024,0.000083,0.000053,0.000244,0.000066,0.000000,0.000081,0.000617,0.000094,0.000220,0.000762,0.001224
2,0.000200,0.000065,0.993807,2.000000e-07,4.700000e-07,0.000073,3.700000e-06,0.000337,6.710000e-06,2.290000e-06,6.380000e-06,7.340000e-06,0.000019,0.000007,3.240000e-06,3.810000e-06,1.411000e-05,1.040000e-06,0.000003,2.300000e-07,1.900000e-07,2.937620e-03,5.200000e-07,9.200000e-07,1.136000e-05,0.000011,3.310400e-04,9.200000e-07,0.000022,8.000000e-08,0.000028,0.000013,5.778000e-05,0.000037,1.683000e-05,6.860000e-06,3.990000e-06,8.030000e-06,1.450000e-06,3.800000e-07,...,0.000083,0.000057,0.000158,0.000073,0.000021,0.000009,0.000002,0.000019,0.000095,0.000021,0.000019,0.000090,0.000073,0.000050,0.000024,0.000000,0.000012,0.000021,0.000000,0.000000,0.000002,0.000000,0.000017,0.000007,0.000002,0.000066,0.000203,0.000026,0.000002,0.000040,0.000007,0.000054,0.000045,0.000000,0.000028,0.000291,0.000033,0.000052,0.000258,0.000215
3,0.791961,0.001496,0.006810,9.390000e-06,1.220000e-06,0.000004,7.452000e-05,0.000003,3.659000e-05,1.630000e-06,8.650000e-06,8.063000e-05,0.003490,0.000009,1.480000e-06,3.260400e-04,1.254000e-05,2.197000e-05,0.000054,2.084000e-05,1.380000e-06,3.500000e-07,2.585000e-05,1.620000e-06,4.539700e-04,0.000008,6.030000e-06,1.475000e-05,0.000015,5.580000e-06,0.000002,0.000024,2.421700e-04,0.006136,5.600000e-06,1.200000e-07,2.860000e-06,4.289000e-05,4.110000e-06,2.330000e-06,...,0.000210,0.000434,0.000543,0.000412,0.000412,0.000045,0.000003,0.000144,0.000282,0.000037,0.000197,0.000218,0.000157,0.000237,0.000021,0.000000,0.000040,0.000056,0.000008,0.000005,0.000013,0.000019,0.000168,0.000013,0.000000,0.000133,0.000202,0.000029,0.000029,0.000035,0.000059,0.001110,0.000075,0.000008,0.000333,0.000793,0.000101,0.000588,0.000503,0.000452
4,0.005782,0.000306,0.004011,1.007000e-05,1.034000e-05,0.000002,3.160000e-06,0.000003,1.984000e-05,5.750000e-06,6.642000e-05,6.690000e-06,0.000301,0.004799,2.800000e-07,1.669000e-05,2.670000e-06,1.060000e-06,0.000011,6.526000e-05,3.100000e-07,3.590000e-05,1.400000e-06,6.260000e-06,1.750103e-02,0.000019,1.190515e-02,4.450000e-06,0.000005,2.387000e-

In [104]:
c3d_hmp_Final.to_pickle('/content/drive/MyDrive/c3d_hmp_Final.pkl')

In [105]:
c3d_hmp_Final = pd.read_pickle('/content/drive/MyDrive/c3d_hmp_Final.pkl')

# Gradient Boost on Final Test Data

In [95]:
X_train = c3d_hmp_Final[0:6000]
X_test = c3d_hmp_Final[6000:]
y_train = gdtruth[['short-term_memorability','long-term_memorability']].values



In [96]:
print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('y_train', y_train.shape)

X_train (6000, 6176)
X_test (2000, 6176)
y_train (6000, 2)


In [97]:
gbr = GradientBoostingRegressor()
model = MultiOutputRegressor(estimator=gbr)
model.fit(X_train,y_train)
gbr_final = model.predict(X_test)


In [98]:
print(gbr_final)

[[0.84662461 0.72596225]
 [0.83719919 0.7492249 ]
 [0.829174   0.72897341]
 ...
 [0.85585285 0.7451828 ]
 [0.84635474 0.77069128]
 [0.87690077 0.76748629]]


# Store results into CSV

In [99]:

print(gb_test_pred)

prediction = pd.DataFrame(gbr_final, columns=['Short-term','long-term']).to_csv('/content/drive/MyDrive/final_result.csv',index=False)



[[0.84662461 0.72596225]
 [0.83719919 0.7492249 ]
 [0.829174   0.72897341]
 ...
 [0.85167743 0.7451828 ]
 [0.84635474 0.77069128]
 [0.87690077 0.76748629]]
